<a href="https://colab.research.google.com/github/romaklym/cnn_image_classfier/blob/master/image_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import tensorflow as tf
import os
from google.colab import drive
import cv2

In [4]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!unzip -u "/content/drive/My Drive/data/train.zip"

Archive:  /content/drive/My Drive/data/train.zip
   creating: data/
  inflating: data/frame10240.jpg     
  inflating: data/frame4966.jpg      
  inflating: data/frame583.jpg       
  inflating: data/frame8184.jpg      
  inflating: data/frame6817.jpg      
  inflating: data/frame11638.jpg     
  inflating: data/frame10526.jpg     
  inflating: data/frame10532.jpg     
  inflating: data/frame6803.jpg      
  inflating: data/frame597.jpg       
  inflating: data/frame8190.jpg      
  inflating: data/frame10254.jpg     
  inflating: data/frame4972.jpg      
  inflating: data/frame6195.jpg      
  inflating: data/frame1822.jpg      
  inflating: data/frame4782.jpg      
  inflating: data/frame11162.jpg     
  inflating: data/frame2495.jpg      
  inflating: data/frame3953.jpg      
  inflating: data/frame11604.jpg     
  inflating: data/frame8806.jpg      
  inflating: data/frame1188.jpg      
  inflating: data/frame8812.jpg      
  inflating: data/frame2481.jpg      
  inflating: data/fr

In [5]:
base_dir = '/content/tensor'

In [6]:
IMAGE_SIZE = 224
BATCH_SIZE = 64

datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

In [7]:
train_generator = datagen.flow_from_directory(
    base_dir,
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    #color_mode='rgb',
    subset='training'
)

val_generator = datagen.flow_from_directory(
    base_dir,
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    subset='validation'
)

Found 9348 images belonging to 1 classes.
Found 2337 images belonging to 1 classes.


In [13]:
print(train_generator.class_indices)
labels = '\n'.join(sorted(
    train_generator.class_indices.keys()))

with open('/content/tensor/labels.txt', 'w') as f:
    f.write(labels)

IMG_SHAPE = (IMAGE_SIZE, IMAGE_SIZE, 3)
base_model = tf.keras.applications.MobileNetV2(
    input_shape=IMG_SHAPE,
    include_top=False,
)

{'data': 0}


In [9]:
base_model.trainable = False
model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(1, activation='softmax')
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [10]:
epochs = 1
history = model.fit(
    train_generator,
    epochs=epochs,
    validation_data=val_generator
)

saved_model_dir = ''
tf.saved_model.save(model, saved_model_dir)

147/147 [==============================] - 707s 5s/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000


INFO:tensorflow:Assets written to: assets


INFO:tensorflow:Assets written to: assets


In [12]:
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
tflite_model = converter.convert()

with open('/content/tensor/model.tflite', 'wb') as f:
    f.write(tflite_model)